In [1]:
import os
import requests, sys
import pandas as pd
from datetime import datetime
import boto3
from botocore.exceptions import NoCredentialsError
import logging
from botocore.exceptions import ClientError
import psycopg2 
# Leer de las variables de entorno
from dotenv import load_dotenv
load_dotenv()


True

In [2]:
AWS_ACCESS_KEY_ID=os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY=os.getenv("AWS_SECRET_ACCESS_KEY")
AWS_DEFAULT_REGION = os.getenv("AWS_DEFAULT_REGION")
AWS_SESSION_TOKEN=os.getenv("AWS_SESSION_TOKEN")
S3_BUCKET_NAME=os.getenv("S3_BUCKET_NAME")
RDS_HOSTNAME = os.getenv("RDS_HOSTNAME")
RDS_USERNAME = os.getenv("RDS_USERNAME")
RDS_PASSWORD = os.getenv("RDS_PASSWORD")
RDS_DBNAME = os.getenv("RDS_DBNAME")
RDS_PORT = os.getenv("RDS_PORT")


In [3]:
# PRUEBA DE CONEXION
s3 = boto3.client(
    's3', 
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    aws_session_token=AWS_SESSION_TOKEN,)
result = s3.get_bucket_acl(Bucket=S3_BUCKET_NAME)
result

{'ResponseMetadata': {'RequestId': '9G9ZRGSH5MDA047Y',
  'HostId': '1T7GvW/fcZsUWz/3rWPFmQkEPsGDmnAsSb57CyfUBeZ9cUUZsyFWhEBXL+QBl3TZVVF7COWLuao=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '1T7GvW/fcZsUWz/3rWPFmQkEPsGDmnAsSb57CyfUBeZ9cUUZsyFWhEBXL+QBl3TZVVF7COWLuao=',
   'x-amz-request-id': '9G9ZRGSH5MDA047Y',
   'date': 'Sat, 19 Oct 2024 19:24:37 GMT',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'Owner': {'DisplayName': 'awslabsc0w4643792t1666354872',
  'ID': '4ec064f8672ee90fab63463ef248c7cf04d0d237ac7de2cbde9409ebca112912'},
 'Grants': [{'Grantee': {'DisplayName': 'awslabsc0w4643792t1666354872',
    'ID': '4ec064f8672ee90fab63463ef248c7cf04d0d237ac7de2cbde9409ebca112912',
    'Type': 'CanonicalUser'},
   'Permission': 'FULL_CONTROL'}]}

In [4]:
import pandas as pd
import psycopg2
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Cargar datos preprocesados desde RDS (PostgreSQL)
def load_from_rds():
    try:
        conn = psycopg2.connect(
            host=RDS_HOSTNAME,  # Reemplaza con tu endpoint de RDS
            user=RDS_USERNAME,  # Reemplaza con tu usuario de RDS
            password=RDS_PASSWORD,  # Reemplaza con tu contraseña de RDS
            dbname=RDS_DBNAME,  # Reemplaza con tu base de datos de RDS
            port=RDS_PORT  # Puerto estándar de PostgreSQL
        )
        query = "SELECT * FROM preprocessed_data"  # Reemplaza con tu tabla de datos preprocesados
        df = pd.read_sql(query, conn)
        conn.close()
        return df
    except Exception as e:
        print(f"Error al cargar los datos desde RDS: {e}")


In [5]:

# Cargar los datos preprocesados desde RDS
data_filtered = load_from_rds()

# Codificar las columnas categóricas si es necesario
le = LabelEncoder()

# Codificar 'clinical_significance' para que sea numérica
data_filtered['clinical_significance'] = le.fit_transform(data_filtered['clinical_significance'])

# Codificar otras columnas categóricas que contengan texto
categorical_columns = ['consequence_type', 'alleles', 'feature_type']
for col in categorical_columns:
    data_filtered[col] = le.fit_transform(data_filtered[col])

# Separar variables independientes (X) y dependiente (y)
X = data_filtered.drop(columns=['clinical_significance'])
y = data_filtered['clinical_significance']

# Dividir el dataset en entrenamiento y prueba (80/20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear y entrenar el modelo Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Predecir en el conjunto de prueba
y_pred = rf_model.predict(X_test)

# Evaluar el modelo
accuracy = accuracy_score(y_test, y_pred)
print(f"Exactitud del modelo: {accuracy:.4f}")
print("Reporte de Clasificación:")
print(classification_report(y_test, y_pred))


C:\Users\lilia\AppData\Local\Temp\ipykernel_8568\3200790176.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


Exactitud del modelo: 0.8790
Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.42      0.36      0.39        78
           1       0.96      0.96      0.96       827
           2       0.66      0.66      0.66        41
           3       0.48      0.56      0.51        54

    accuracy                           0.88      1000
   macro avg       0.63      0.63      0.63      1000
weighted avg       0.88      0.88      0.88      1000



In [13]:

# Guardar resultados de predicciones en RDS (PostgreSQL)
def save_predictions_to_rds(X_test, predictions):
    try:
        conn = psycopg2.connect(
            host=RDS_HOSTNAME,  # Reemplaza con tu endpoint de RDS
            user=RDS_USERNAME,  # Reemplaza con tu usuario de RDS
            password=RDS_PASSWORD,  # Reemplaza con tu contraseña de RDS
            dbname=RDS_DBNAME,  # Reemplaza con tu base de datos de RDS
            port=RDS_PORT  # Puerto estándar de PostgreSQL
        )
        cursor = conn.cursor()
        for idx, pred in enumerate(predictions):
            #int_variation_id=X_test.index[idx]
            #int_prediction=(int)pred

            cursor.execute("INSERT INTO predictions (variation_id, prediction) VALUES (%s, %s)",
                           (X_test.index[idx].item(),pred.item()))
        conn.commit()
        cursor.close()
        conn.close()
        print("Predicciones guardadas exitosamente en RDS.")
    except Exception as e:
        print(f"Error al guardar las predicciones en RDS: {e}")

# Guardar las predicciones en RDS
save_predictions_to_rds(X_test, y_pred)



Predicciones guardadas exitosamente en RDS.


In [ ]:
CREATE TABLE preprocessed_data (
    consequence_type VARCHAR(255),
    clinical_significance VARCHAR(255),
    feature_type VARCHAR(255),
    start INT,
    "end" INT,
    strand INT,
    alleles VARCHAR(1000)
);

CREATE TABLE predictions (
    variation_id INT,
    prediction INT
);


In [ ]:
# Dockerfile for preprocessing data and saving to S3

# Use an official Python runtime as a parent image
FROM python:3.9-slim

# Set the working directory in the container
WORKDIR /usr/src/app

# Copy the current directory contents into the container at /usr/src/app
COPY . .

# Install any necessary dependencies
RUN pip install --no-cache-dir pandas boto3

# Command to run the script
CMD ["python", "./preprocessing_to_s3.py"]


In [ ]:
# Dockerfile for loading data from S3, training a model, and saving predictions to RDS

# Use an official Python runtime as a parent image
FROM python:3.9-slim

# Set the working directory in the container
WORKDIR /usr/src/app

# Copy the current directory contents into the container at /usr/src/app
COPY . .

# Install any necessary dependencies
RUN pip install --no-cache-dir pandas scikit-learn mysql-connector-python boto3

# Command to run the script
CMD ["python", "./modeling_from_s3_to_rds.py"]
